______________________________________
# <center>**Trabajo Practico Nº1 para la Materia *Organización de Datos***</center>

*Integrantes*:
- 103963	Carolina Di Matteo	cdimatteo@fi.uba.ar
- 101231	Pablo Salvador Dimartino	pdimartino@fi.uba.ar
- 100113	Juan Sebastian Burgos	jsburgos@fi.uba.ar
- 104415	Valentina Laura Correa	vcorrea@fi.uba.ar

*Grupo*: 14

*Repositorio*: [github](https://github.com/valencorrea/7506R-2C2022-GRUPO14)

*Curso*: Rodriguez

*Cuatrimestre*: 2c2022

Datos provistos por [properati](https://www.properati.com.ar).
______________________________________


In [1]:
#Importación de librerías

import numpy as np
import cmath as math
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from xgboost import XGBRegressor

# from sklearn.model_selection import StratifiedKFold, KFold, RandomizedSearchCV, train_test_split

#Configuración de Warnings
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

#Ejecución con Drive
if 'google.colab' in str(get_ipython()):
    IN_COLAB = True
else:
    IN_COLAB = False

if IN_COLAB :
    from google.colab import drive
    drive.mount('/content/drive')
    properati = pd.read_csv('/content/drive/MyDrive/📔 Organización de Datos (75.06)/TPS/TP1/1d_df_reducido.csv')
    properati_decrip = pd.read_csv('/content/drive/MyDrive/📔 Organización de Datos (75.06)/TPS/TP1/properati_argentina_2021_decrip.csv')
else:
    properati=pd.read_csv('./1d_df_reducido.csv')
    properati_decrip = pd.read_csv('properati_argentina_2021_decrip.csv')

In [2]:
properati

,id,start_date,end_date,latitud,longitud,place_l4,property_type,property_rooms,property_surface_total,property_price
0,1gMQ4rXcpnPs+hhvW11Jdw==,738005,738091,-34.622385,-58.448886,Caballito,Departamento,4.0,113.0,288000.0
1,p7gmyqzdOy/+odHwH5oPyg==,737817,737946,-34.618084,-58.359912,Puerto Madero,Departamento,2.0,70.0,450000.0
2,ZYkS+4dCz1vUENzAREcX9w==,738035,738035,-34.601159,-58.398106,Balvanera,Departamento,1.0,33.0,77243.0
3,UL/s8zoicLDXXwhaelBgFQ==,738140,738195,-34.605542,-58.390007,San Nicolás,Departamento,1.0,30.0,65000.0
4,wnz+hkCE7RVBNcqDQEtFSw==,737868,737946,-34.593266,-58.381426,Retiro,Departamento,1.0,40.0,79000.0
...,...,...,...,...,...,...,...,...,...,...
74248,CIrzzz0KQkUDDmjnFINLwA==,738001,738192,-34.597304,-58.432406,Villa Crespo,Departamento,1.0,29.0,64500.0
74249,i5lDg1C7ZEZyETXxCbolhw==,737812,737971,-34.581910,-58.449381,Palermo,Departamento,2.0,56.0,144870.0
74250,b97ekVIR18ZDn3ZsDXWrVQ==,738096,738182,-34.604510,-58.390908,Villa Crespo,Departamento,3.0,64.0,110000.0
74251,7oLMN+ZSoOP2N9Uu7Pz68A==,737829,737856,-34.657111,-58.405545,Pompeya,Departamento,4.0,81.0,73000.0


## 1.Procesamiento del Lenguaje Natural
### 1.a Ampliación del dataset

In [3]:
# df_decrip = properati_decrip.head(10000).copy()
df_decrip = pd.merge(properati, properati_decrip, on="id")
df_decrip = df_decrip[["id", "property_description"]]
df_decrip['property_description'].isna().sum()
df_decrip

,id,property_description
0,1gMQ4rXcpnPs+hhvW11Jdw==,"ESPECTACULAR 4 AMBIENTES, DORMITORIO SUITE, BA..."
1,p7gmyqzdOy/+odHwH5oPyg==,"Excelente departamento, piso alto con una herm..."
2,ZYkS+4dCz1vUENzAREcX9w==,Corredor Responsable: Gustavo Guastello - C.U....
3,UL/s8zoicLDXXwhaelBgFQ==,Corredor Responsable: GRACIELA MENAZZI CMCPDJL...
4,wnz+hkCE7RVBNcqDQEtFSw==,Muy lindo y funcional departamento de un ambie...
...,...,...
74248,CIrzzz0KQkUDDmjnFINLwA==,MONOAMBIENTE DIVIDIDO. FRENTE. 29 M2. IMPECABL...
74249,i5lDg1C7ZEZyETXxCbolhw==,Departamento. Edificio OM Palermo. Living/Come...
74250,b97ekVIR18ZDn3ZsDXWrVQ==,Tres ambientes en Balvanera reciclado a nuevo!...
74251,7oLMN+ZSoOP2N9Uu7Pz68A==,Corredor Responsable: Micaela Perez / Lucas Fe...


In [4]:
df_decrip.dropna(inplace=True)

In [5]:
from collections import Counter
Counter(" ".join(df_decrip["property_description"]).split()).most_common(100)


[('de', 858011),
 ('y', 507798),
 ('con', 401876),
 ('la', 290201),
 ('en', 278181),
 ('a', 243761),
 ('el', 169956),
 ('del', 153280),
 ('que', 140058),
 ('por', 133636),
 ('al', 125215),
 ('-', 113886),
 ('un', 103720),
 ('las', 90713),
 ('los', 88438),
 ('para', 88196),
 ('se', 63111),
 ('DE', 56458),
 ('son', 56422),
 ('2', 52793),
 ('es', 50385),
 ('una', 48933),
 ('3', 41260),
 ('cocina', 38889),
 ('ambientes', 38850),
 ('valor', 38421),
 ('esta', 38294),
 ('muy', 36893),
 ('x', 36800),
 ('Av.', 36631),
 ('comedor', 35535),
 ('baño', 35477),
 ('Y', 34142),
 ('CON', 33915),
 ('no', 33913),
 ('piso', 33349),
 ('o', 33053),
 ('/', 32255),
 ('tu', 32102),
 ('A', 32063),
 ('hasta', 30270),
 ('balcón', 29805),
 ('inmueble', 28725),
 ('casa', 28656),
 ('No', 28636),
 ('propiedad.', 28071),
 ('30%', 27685),
 ('departamento', 27671),
 ('EN', 27267),
 ('cuadras', 27071),
 ('Corredor', 26992),
 ('préstamo', 26921),
 ('cuota', 26781),
 ('medidas', 26423),
 ('Responsable:', 25971),
 ('living'

In [6]:
df_decrip["property_description"]

0        ESPECTACULAR 4 AMBIENTES, DORMITORIO SUITE, BA...
1        Excelente departamento, piso alto con una herm...
2        Corredor Responsable: Gustavo Guastello - C.U....
3        Corredor Responsable: GRACIELA MENAZZI CMCPDJL...
4        Muy lindo y funcional departamento de un ambie...
                               ...                        
74248    MONOAMBIENTE DIVIDIDO. FRENTE. 29 M2. IMPECABL...
74249    Departamento. Edificio OM Palermo. Living/Come...
74250    Tres ambientes en Balvanera reciclado a nuevo!...
74251    Corredor Responsable: Micaela Perez / Lucas Fe...
74252    Monoambiente en 8vo piso al contrafrente, todo...
Name: property_description, Length: 74253, dtype: object

In [7]:
df_decrip["property_description"] = df_decrip["property_description"].apply(lambda line: line.replace("<br>", " ")) # remover etiqueta de html
df_decrip["property_description"] = df_decrip["property_description"].apply(lambda line: line.lower()) # Todo a minúsculas
import unidecode
df_decrip["property_description"] = df_decrip["property_description"].apply(lambda line: unidecode.unidecode(line)) # Sin tildes por posibles errores de gramática

In [8]:
import re
# df_decrip["property_description"] = df_decrip["property_description"].apply(lambda line: re.sub("[^\\w-]+", " ", line)) #Remueve simbolos menos el -
df_decrip["property_description"] = df_decrip["property_description"].apply(lambda line: re.sub(r'[^\w]', ' ', line)) # remover símbolos

In [9]:
df_decrip["property_description"] = df_decrip["property_description"].apply(lambda line: re.sub("\s\s+" , " ", line)) # remover multiples espacios para facilitar lectura

In [10]:
# export = df_decrip["property_description"].head()
# np.savetxt('export.txt', export.values, fmt='%s')

In [11]:
df_nostopwords = df_decrip.copy()

with open('stopwords') as f:
    lines = f.read().splitlines()

f = lambda x: ' '.join([item for item in x.split() if item not in lines]) #remueve stopwords
df_nostopwords["property_description"] = df_nostopwords["property_description"].apply(f)

df_nostopwords["property_description"].head(10)


0    espectacular 4 ambientes dormitorio suite balc...
1    excelente departamento piso alto hermosa vista...
2    corredor responsable gustavo guastello c u c i...
3    corredor responsable graciela menazzi cmcpdjlm...
4    lindo funcional departamento ambiente distribu...
5    corredor responsable mario gabriel gerez cmpcs...
6    corredor responsable ariel champanier andrea b...
7    hermoso 2 amb edificio black complejo musa sta...
8    corredor responsable daniel acosta cucicba 240...
9    excelente 3 ambientes torre aisenson almagro e...
Name: property_description, dtype: object

In [12]:
from collections import Counter
Counter(" ".join(df_nostopwords["property_description"]).split()).most_common(100)

[('cocina', 89461),
 ('2', 88206),
 ('bano', 74431),
 ('3', 74276),
 ('ambientes', 71519),
 ('balcon', 71288),
 ('comedor', 68491),
 ('piso', 67831),
 ('living', 63838),
 ('propiedad', 62563),
 ('departamento', 54477),
 ('edificio', 53423),
 ('av', 50589),
 ('1', 50145),
 ('dormitorio', 48863),
 ('completo', 48392),
 ('pisos', 46213),
 ('4', 46048),
 ('frente', 44714),
 ('expensas', 44287),
 ('n', 44025),
 ('excelente', 41316),
 ('corredor', 41146),
 ('medidas', 41048),
 ('x', 40904),
 ('inmueble', 40569),
 ('responsable', 37724),
 ('amplio', 36356),
 ('m2', 34044),
 ('30', 34028),
 ('placard', 33950),
 ('casa', 32678),
 ('dormitorios', 32258),
 ('lavadero', 31877),
 ('c', 31791),
 ('cuadras', 31167),
 ('luminoso', 30549),
 ('cucicba', 30414),
 ('accede', 29383),
 ('terraza', 29330),
 ('ley', 27849),
 ('aire', 27700),
 ('lendar', 27450),
 ('queres', 27417),
 ('prestamo', 27367),
 ('vista', 26980),
 ('venta', 26867),
 ('cuota', 26857),
 ('podes', 26581),
 ('salida', 26111),
 ('ubicacion

Seleccionamos los aspectos que nos parecen relevantes, para luego buscar sus posibles valores.
Los aspectos elegidos son: Cocina, pisos, calefaccion, expensas, lavadero, balcon,  cochera y aire.

In [13]:
from types import NoneType
aspectos = ['cocina', 'pisos', 'calefaccion', 'expensas', 'lavadero', 'balcon', 'cochera', 'aire']

def f(line, word):
    format = r"\W*([\w]+)"
    n = 2
    x = re.search(r'{}\W*{}{}'.format(format*n, word, format*n), line)
    if not isinstance(x, NoneType):
        return x.group()
    else:
        return ""

for word in aspectos:
    df_nostopwords[word] = df_nostopwords["property_description"].apply(lambda line: f(line, word))

In [14]:
cant_val_aspectos = 15

In [15]:
Counter(" ".join(df_nostopwords["cocina"]).split()).most_common(cant_val_aspectos)

[('cocina', 62811),
 ('comedor', 19199),
 ('integrada', 11530),
 ('living', 9979),
 ('bano', 8949),
 ('lavadero', 6978),
 ('balcon', 6225),
 ('independiente', 6077),
 ('completo', 5385),
 ('diario', 4993),
 ('muebles', 4829),
 ('separada', 4746),
 ('amplia', 4518),
 ('toilette', 3963),
 ('completa', 3692)]

Para Cocina, posibles valores son: integrada, separada, independiente, completa

In [16]:
Counter(" ".join(df_nostopwords["pisos"]).split()).most_common(cant_val_aspectos)

[('pisos', 31830),
 ('parquet', 6394),
 ('madera', 4427),
 ('porcelanato', 4025),
 ('living', 3671),
 ('edificio', 3624),
 ('comedor', 3155),
 ('unidades', 1821),
 ('cocina', 1754),
 ('departamentos', 1662),
 ('bano', 1576),
 ('2', 1411),
 ('4', 1314),
 ('3', 1241),
 ('ambientes', 1201)]

Para pisos, posibles valores son: porcelanato, parquet, madera, ceramicos

In [17]:
Counter(" ".join(df_nostopwords["calefaccion"]).split()).most_common(cant_val_aspectos)

[('calefaccion', 15450),
 ('radiante', 4448),
 ('losa', 3916),
 ('central', 3444),
 ('caliente', 2653),
 ('agua', 2257),
 ('radiadores', 2052),
 ('individual', 1921),
 ('aire', 1777),
 ('piso', 1527),
 ('caldera', 1246),
 ('tiro', 1197),
 ('acondicionado', 1097),
 ('servicios', 1056),
 ('ambientes', 798)]

Para calefaccion, posibles valores son: radiadores, radiante, central, individual

In [18]:
Counter(" ".join(df_nostopwords["expensas"]).split()).most_common(cant_val_aspectos)

[('expensas', 35165),
 ('servicios', 6846),
 ('impuestos', 5906),
 ('valores', 5559),
 ('bajas', 4879),
 ('funcionales', 4365),
 ('000', 3296),
 ('abl', 2445),
 ('medidas', 1959),
 ('aysa', 1606),
 ('sujetos', 1542),
 ('propiedad', 1276),
 ('superficies', 1260),
 ('indicados', 1242),
 ('consignadas', 1189)]

Para expensas, posibles valores son: bajas, incluye

In [19]:
Counter(" ".join(df_nostopwords["lavadero"]).split()).most_common(cant_val_aspectos)

[('lavadero', 28549),
 ('independiente', 7972),
 ('cocina', 7923),
 ('comedor', 4376),
 ('bano', 4342),
 ('diario', 3175),
 ('servicio', 2922),
 ('dependencia', 2920),
 ('incorporado', 2506),
 ('completo', 2102),
 ('patio', 1882),
 ('separado', 1807),
 ('balcon', 1738),
 ('espacio', 1736),
 ('toilette', 1385)]

Para lavadero, posibles valores son: independiente, separado

In [20]:
Counter(" ".join(df_nostopwords["balcon"]).split()).most_common(cant_val_aspectos)

[('balcon', 42917),
 ('salida', 9996),
 ('frente', 9711),
 ('comedor', 8052),
 ('ambientes', 6387),
 ('corrido', 6126),
 ('cocina', 4569),
 ('living', 4207),
 ('vista', 4051),
 ('amplio', 3716),
 ('terraza', 3682),
 ('aterrazado', 3172),
 ('2', 3151),
 ('luminoso', 2855),
 ('3', 2684)]

Para balcon, posibles valores son: frente, amplio, terraza, salida, corrido

In [21]:
Counter(" ".join(df_nostopwords["cochera"]).split()).most_common(cant_val_aspectos*2)

[('cochera', 17208),
 ('cocheras', 7359),
 ('fija', 4756),
 ('cubierta', 4118),
 ('ambientes', 3484),
 ('baulera', 2863),
 ('2', 2467),
 ('edificio', 2119),
 ('balcon', 1521),
 ('3', 1399),
 ('opcional', 1307),
 ('1', 1132),
 ('fijas', 1080),
 ('terraza', 1007),
 ('dependencia', 979),
 ('posibilidad', 958),
 ('piso', 826),
 ('frente', 780),
 ('bano', 760),
 ('departamento', 759),
 ('disponibles', 759),
 ('4', 753),
 ('servicio', 729),
 ('planta', 701),
 ('parrilla', 699),
 ('espacio', 615),
 ('subsuelo', 598),
 ('amenities', 586),
 ('completo', 553),
 ('patio', 525)]

Para cochera, posibles valores son: cubierta, descubierta, opcional

In [22]:
Counter(" ".join(df_nostopwords["aire"]).split()).most_common(cant_val_aspectos)

[('aire', 20578),
 ('acondicionado', 14337),
 ('frio', 5985),
 ('aires', 5809),
 ('split', 2204),
 ('ciudad', 2020),
 ('acondicionados', 1961),
 ('luz', 1851),
 ('equipos', 1585),
 ('calefaccion', 1493),
 ('instalacion', 1413),
 ('ambientes', 1411),
 ('living', 1138),
 ('central', 1062),
 ('balcon', 1002)]

Para aire, posibles valores son: split, central, acondicionado

Teniendo en cuenta los posibles valores para los aspectos considerados, decidimos descartar lavadero.
De esta forma, nuestros aspectos serán: Cocina, pisos, calefaccion, expensas, balcon,  cochera y aire.

In [23]:
aspectos = ['cocina', 'pisos', 'calefaccion', 'expensas', 'balcon', 'cochera', 'aire']

A continuación creamos la variable values, que contiene los posibles valores para cada uno de los aspectos elegidos.

In [24]:
values_cocina = ['integrada', 'separada', 'independiente', 'completa']
values_pisos = ['porcelanato', 'madera', 'parquet', 'ceramicos']
values_calefaccion = ['radiadores', 'radiante', 'central', 'individual']
values_expensas = ['bajas', 'incluye']
values_balcon = ['frente', 'amplio', 'terraza', 'salida', 'corrido']
values_cochera = ['opcional', 'cubierta', 'descubierta']
values_aire = ['split', 'central', 'acondicionado']

values = [values_cocina, values_pisos, values_calefaccion, values_expensas, values_balcon, values_cochera, values_aire]

En primer lugar creo un dataframe auxiliar que tenga los IDs y las columnas de aspectos

In [25]:
aux_df = df_nostopwords.copy()
aux_df.drop('property_description', inplace=True, axis=1)

Luego creamos una función a la que; pasandole un dataframe, los aspectos y el listado de valores posibles, reemplace el contenido de las columnas por los valores correspondientes

In [26]:
def limpiar_values_de_aspects(df, aspects, values):
    i = 0
    for aspect in aspects:
        for word in values[i]:
            df[aspect] = df[aspect].apply(lambda line: word if word in line else line)
        df[aspect] = df[aspect].apply(lambda line: line if len(line.split())<2 else '')
        i = i+1

Ahora modificamos las columnas de los aspectos, para que solo queden los valores.

In [27]:
limpiar_values_de_aspects(aux_df, aspectos, values)
aux_df.drop("lavadero", axis=1, inplace=True)
aux_df.head(20)

,id,cocina,pisos,calefaccion,expensas,balcon,cochera,aire
0,1gMQ4rXcpnPs+hhvW11Jdw==,,,,,terraza,opcional,
1,p7gmyqzdOy/+odHwH5oPyg==,,,,,amplio,,
2,ZYkS+4dCz1vUENzAREcX9w==,,,,,,,
3,UL/s8zoicLDXXwhaelBgFQ==,separada,,,,,,acondicionado
4,wnz+hkCE7RVBNcqDQEtFSw==,,,,,,,
5,+QvrIZqpMPRxlz+ilj1LkA==,integrada,,,,,,
6,MGM2azMohIovCwT8jcExeQ==,,,,,,,
7,uLDfFrhFSDasD1UObhqHmg==,independiente,,,,frente,,acondicionado
8,6iHL1ay2QcrLeLh+K33ziw==,,,,bajas,,,
9,8J08zxRGfHVQ3dzTyopnWQ==,independiente,,radiante,,salida,,


Por último hacemos el merge con el dataframe original, teniendo en cuenta los ID

In [28]:
df = properati.copy()
df = pd.merge(df,aux_df, on="id")
df

,id,start_date,end_date,latitud,longitud,place_l4,property_type,property_rooms,property_surface_total,property_price,cocina,pisos,calefaccion,expensas,balcon,cochera,aire
0,1gMQ4rXcpnPs+hhvW11Jdw==,738005,738091,-34.622385,-58.448886,Caballito,Departamento,4.0,113.0,288000.0,,,,,terraza,opcional,
1,p7gmyqzdOy/+odHwH5oPyg==,737817,737946,-34.618084,-58.359912,Puerto Madero,Departamento,2.0,70.0,450000.0,,,,,amplio,,
2,ZYkS+4dCz1vUENzAREcX9w==,738035,738035,-34.601159,-58.398106,Balvanera,Departamento,1.0,33.0,77243.0,,,,,,,
3,UL/s8zoicLDXXwhaelBgFQ==,738140,738195,-34.605542,-58.390007,San Nicolás,Departamento,1.0,30.0,65000.0,separada,,,,,,acondicionado
4,wnz+hkCE7RVBNcqDQEtFSw==,737868,737946,-34.593266,-58.381426,Retiro,Departamento,1.0,40.0,79000.0,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74248,CIrzzz0KQkUDDmjnFINLwA==,738001,738192,-34.597304,-58.432406,Villa Crespo,Departamento,1.0,29.0,64500.0,,parquet,,,,,
74249,i5lDg1C7ZEZyETXxCbolhw==,737812,737971,-34.581910,-58.449381,Palermo,Departamento,2.0,56.0,144870.0,,,,,,,
74250,b97ekVIR18ZDn3ZsDXWrVQ==,738096,738182,-34.604510,-58.390908,Villa Crespo,Departamento,3.0,64.0,110000.0,independiente,,,,,,acondicionado
74251,7oLMN+ZSoOP2N9Uu7Pz68A==,737829,737856,-34.657111,-58.405545,Pompeya,Departamento,4.0,81.0,73000.0,,,,,,,


### 1.b Modelos

Primero entrenamos un modelo sin las nuevas columnas como en el TP1

In [55]:
df_TP1= pd.get_dummies(properati, columns=['place_l4', 'property_type'], drop_first=True)
train = df_TP1.sample(frac=0.8, random_state=200)
y_train = train["property_price"]
train.drop(["property_price", "id"], inplace=True, axis=1)
test = df_TP1.drop(train.index)
y_test = test["property_price"]
test.drop(["property_price", "id"], inplace=True, axis=1)

25158     82000.0
23761    128000.0
442      189000.0
67780    148000.0
58586    149000.0
           ...   
45547    155000.0
2931     169000.0
49368     89000.0
37511    345000.0
62855    140000.0
Name: property_price, Length: 59402, dtype: float64

In [57]:
param_grid = {'min_child_weight': 5, 'max_depth': 4, 'learning_rate': 0.15, 'gamma': 0.4, 'colsample_bytree': 0.5}
xgb_regressor = XGBRegressor(**param_grid).fit(train, y_train)
predictions = xgb_regressor.predict(test)
xgb_regressor.score(test, y_test)

0.8541233081740065

0.8541233081740065

Ahora entrenamos un modelo con las nuevas columnas.

Realizamos one hot encoding para las variables cualitativas:

In [31]:
df_dummies = pd.get_dummies(df, columns=['place_l4', 'property_type', 'cocina', 'pisos', 'calefaccion', 'expensas', 'balcon', 'cochera', 'aire'], drop_first=True)
df_dummies

,id,start_date,end_date,latitud,longitud,property_rooms,property_surface_total,property_price,place_l4_Agronomía,place_l4_Almagro,...,balcon_amplio,balcon_corrido,balcon_frente,balcon_salida,balcon_terraza,cochera_cubierta,cochera_opcional,aire_acondicionado,aire_central,aire_split
0,1gMQ4rXcpnPs+hhvW11Jdw==,738005,738091,-34.622385,-58.448886,4.0,113.0,288000.0,0,0,...,0,0,0,0,1,0,1,0,0,0
1,p7gmyqzdOy/+odHwH5oPyg==,737817,737946,-34.618084,-58.359912,2.0,70.0,450000.0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,ZYkS+4dCz1vUENzAREcX9w==,738035,738035,-34.601159,-58.398106,1.0,33.0,77243.0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,UL/s8zoicLDXXwhaelBgFQ==,738140,738195,-34.605542,-58.390007,1.0,30.0,65000.0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,wnz+hkCE7RVBNcqDQEtFSw==,737868,737946,-34.593266,-58.381426,1.0,40.0,79000.0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74248,CIrzzz0KQkUDDmjnFINLwA==,738001,738192,-34.597304,-58.432406,1.0,29.0,64500.0,0,0,...,0,0,0,0,0,0,0,0,0,0
74249,i5lDg1C7ZEZyETXxCbolhw==,737812,737971,-34.581910,-58.449381,2.0,56.0,144870.0,0,0,...,0,0,0,0,0,0,0,0,0,0
74250,b97ekVIR18ZDn3ZsDXWrVQ==,738096,738182,-34.604510,-58.390908,3.0,64.0,110000.0,0,0,...,0,0,0,0,0,0,0,1,0,0
74251,7oLMN+ZSoOP2N9Uu7Pz68A==,737829,737856,-34.657111,-58.405545,4.0,81.0,73000.0,0,0,...,0,0,0,0,0,0,0,0,0,0


Separamos el dataset en train y test

In [44]:

train = df_dummies.sample(frac=0.8, random_state=200)
y_train = train["property_price"]
train.drop(["property_price", "id"], inplace=True, axis=1)
test = df_dummies.drop(train.index)
y_test = test["property_price"]
test.drop(["property_price", "id"], inplace=True, axis=1)
# train
y_train

25158     82000.0
23761    128000.0
442      189000.0
67780    148000.0
58586    149000.0
           ...   
45547    155000.0
2931     169000.0
49368     89000.0
37511    345000.0
62855    140000.0
Name: property_price, Length: 59402, dtype: float64

In [49]:
from xgboost import XGBRegressor
param_grid = {'min_child_weight': 5, 'max_depth': 4, 'learning_rate': 0.15, 'gamma': 0.4, 'colsample_bytree': 0.5}
xgb_regressor = XGBRegressor(**param_grid).fit(train, y_train)

In [51]:
predictions = xgb_regressor.predict(test)
predictions

array([ 39512.836, 112741.38 , 326897.84 , ..., 239910.23 ,  49005.348,
       151297.84 ], dtype=float32)

In [54]:
xgb_regressor.score(test,y_test)

0.8605136268044646

Vemos que mejora un poco el score con las nuevas columnas.

Ahora optimizamos los hiperparámetros con el nuevo dataset ampliado:

In [59]:
import sklearn as sk
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score

params_grid = {'learning_rate': [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ],
               'max_depth': [3, 4, 5, 6, 8, 10, 12, 15],
               'min_child_weight': [1, 3, 5, 7],
               'gamma': [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
               'colsample_bytree' : [ 0.3, 0.4, 0.5 , 0.7 ]}

randomCV = RandomizedSearchCV(estimator = XGBRegressor(),
                              param_distributions = params_grid,
                              scoring = make_scorer(accuracy_score),
                              cv = StratifiedKFold(n_splits = 5),
                              n_iter = 5)

randomCV.fit(train, y_train)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          enable_categorical=False, gamma=None,
                                          gpu_id=None, importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=No...
                                          reg_alpha=None, reg_lambda=None,
                                          scale_pos_weight=None, subsample=None,
                                          tree_method=None,
                                          validate_parameters=None,
                                          verbosity=None),
                   n_iter=5,
                   param_distributions={'colsample_bytree': [0.3, 0.4, 0.5,
                                                             0.7],
                                        'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
                                        'learning_rate': [0.05, 0.1, 0.15, 0.2,
                                                          0.25, 0.3],
                                        'max_depth': [3, 4, 5, 6, 8, 10, 12,
                                                      15],
                                        'min_child_weight': [1, 3, 5, 7]},
                   scoring=make_scorer(accuracy_score))

In [63]:
best_estimator = randomCV.best_estimator_
best_estimator.score(test,y_test)

0.8747254649095566

Observamos que el score mejoró levemente

## 2.Redes Neuronales
### 2.a Regresión

Generamos la variable target tipo_precio

In [65]:
df_trabajo = properati.copy()
df_trabajo["property_m2price"] = df_trabajo["property_price"] / df_trabajo["property_surface_total"]
df_trabajo.sort_values(by=["property_m2price"], inplace=True)

df_trabajo['tipo_precio_3'] = pd.qcut(x=df_trabajo.loc[df_trabajo['property_type'] == 'Departamento']['property_m2price'], q=4, labels=['bajo', 'medio', 'medio2', 'alto']).replace("medio2","medio")
df_trabajo['tipo_precio_3'].fillna(pd.qcut(x=df_trabajo.loc[df_trabajo['property_type'] == 'Casa']['property_m2price'], q=4, labels=['bajo', 'medio', 'medio2', 'alto']).replace("medio2","medio"), inplace=True)
df_trabajo['tipo_precio_3'].fillna(pd.qcut(x=df_trabajo.loc[df_trabajo['property_type'] == 'PH']['property_m2price'], q=4, labels=['bajo', 'medio', 'medio2', 'alto']).replace("medio2","medio"), inplace=True)

In [66]:
df_trabajo= pd.get_dummies(df_trabajo, columns=['place_l4', 'property_type'], drop_first=True)
train = df_trabajo.sample(frac=0.8, random_state=200)
y_train = train["tipo_precio_3"]
train.drop(["property_price", "id", "tipo_precio_3", "property_m2price"], inplace=True, axis=1)
test = df_trabajo.drop(train.index)
y_test = test["tipo_precio_3"]
test.drop(["property_price", "id", "tipo_precio_3", "property_m2price"], inplace=True, axis=1)

In [64]:
from sklearn.metrics import recall_score, accuracy_score, f1_score, precision_score
def metricas(y_test, y_pred):
    print(f'Accuracy: {round(accuracy_score(y_test, y_pred),2)}')
    print(f'Precision: {round(precision_score(y_test, y_pred, average="macro"),2)}')
    print(f'Recall: {round(recall_score(y_test, y_pred, average="macro"),2)}')
    print(f'F1 Score: {round(f1_score(y_test, y_pred, average="macro"),2)}')

In [ ]:
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from keras.utils.vis_utils import plot_model
import visualkeras